In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import collections
import pickle

import copy 
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tff.backends.reference.set_reference_context()

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import models

import math
import pickle
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from operator import add

import seaborn as sns
sns.set_style("whitegrid")

In [4]:
D = pickle.load(open('COUNT','rb'))
FROUND = pickle.load(open('R1','rb'))
OROUND = pickle.load(open('Rs','rb'))

In [5]:
def getModel():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(10,activation="sigmoid")
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [6]:
import numpy as np
from collections import Counter

def getweight(df):
    LBL = Counter(df.digit)
    maen_arr = np.mean(list(map(lambda x:x[1]/D[x[0]],list(LBL.items()))))
    return maen_arr

In [7]:
writer_models = []
cw = []

def train_on_clients():
    for writer in range(1,31):
        df = FROUND.get(writer)
        cw.append(getweight(df))
        labels = df['digit'].to_numpy()
        df_copy = df.drop(['digit','writer'], axis=1)
        model = getModel()
        model.fit(df_copy.to_numpy(), labels, epochs=10)
        writer_models.append(model)

In [8]:
train_on_clients()

Epoch 1/10
2/2 [==============================] - 0s 3ms/step - loss: 58.8319 - accuracy: 0.2300
Epoch 2/10
2/2 [==============================] - 0s 3ms/step - loss: 57.6283 - accuracy: 0.1987
Epoch 3/10
2/2 [==============================] - 0s 3ms/step - loss: 54.6514 - accuracy: 0.2063
Epoch 4/10
2/2 [==============================] - 0s 3ms/step - loss: 52.6841 - accuracy: 0.2092
Epoch 5/10
2/2 [==============================] - 0s 2ms/step - loss: 47.4975 - accuracy: 0.2433
Epoch 6/10
2/2 [==============================] - 0s 2ms/step - loss: 44.6500 - accuracy: 0.2433
Epoch 7/10
2/2 [==============================] - 0s 2ms/step - loss: 42.8207 - accuracy: 0.2433
Epoch 8/10
2/2 [==============================] - 0s 3ms/step - loss: 41.9499 - accuracy: 0.2121
Epoch 9/10
2/2 [==============================] - 0s 2ms/step - loss: 40.3845 - accuracy: 0.2329
Epoch 10/10
2/2 [==============================] - 0s 3ms/step - loss: 37.0143 - accuracy: 0.2433
Epoch 1/10
2/2 [=============

2/2 [==============================] - 0s 2ms/step - loss: 18.8732 - accuracy: 0.2196
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 18.7075 - accuracy: 0.1883
Epoch 9/10
2/2 [==============================] - 0s 3ms/step - loss: 17.9669 - accuracy: 0.2196
Epoch 10/10
2/2 [==============================] - 0s 2ms/step - loss: 16.6609 - accuracy: 0.1779
Epoch 1/10
2/2 [==============================] - 0s 3ms/step - loss: 47.8749 - accuracy: 0.0846
Epoch 2/10
2/2 [==============================] - 0s 2ms/step - loss: 44.8873 - accuracy: 0.0950
Epoch 3/10
2/2 [==============================] - 0s 2ms/step - loss: 42.6618 - accuracy: 0.0742
Epoch 4/10
2/2 [==============================] - 0s 3ms/step - loss: 40.5477 - accuracy: 0.0846
Epoch 5/10
2/2 [==============================] - 0s 3ms/step - loss: 37.5969 - accuracy: 0.0846
Epoch 6/10
2/2 [==============================] - 0s 2ms/step - loss: 36.2735 - accuracy: 0.0846
Epoch 7/10
2/2 [========================

In [9]:
 def average(rounds):
        agg_model = rounds[0]
        l1,b1,l2,b2 = agg_model.get_weights()
        l1 = l1
        b1 = b1
        l2 = l2
        b2 = b2
        for i in rounds[1:]:
            tl1,tb1,tl2,tb2 = i.get_weights()
            l1+=tl1
            b1+=tb1        
            l2+=tl2        
            b2+=tb2
        agg_model.set_weights([l1/30,b1/30,l2/30,b2/30])
        return agg_model

In [10]:
DF_TEST = pd.read_csv('TEST.csv').iloc[:,1:]
labels_tes = DF_TEST['digit'].to_numpy()
df_tes = DF_TEST.drop(['digit'], axis=1)

In [11]:
agg_model = average(writer_models)

In [12]:
agg_model.evaluate(df_tes.to_numpy(), labels_tes)

110/110 [==============================] - 0s 1ms/step - loss: 2.6115 - accuracy: 0.0537


[2.6115267276763916, 0.05374499782919884]

In [13]:
from tqdm.notebook import tqdm
GLOBAL = []
history = []
for i in tqdm(range(20)):
    rounds = []
    cw_local = []
    for writer in range(1,31):
        df = OROUND[i].get(writer)
        cw_local.append(getweight(df))
        labels = df['digit'].to_numpy()
        df_copy = df.drop(['digit','writer'], axis=1)
        model = agg_model
        model.fit(df_copy.to_numpy(), labels, epochs=10,verbose=1)
        rounds.append(model)
    agg_model = average(rounds)
    GLOBAL.append({
        'round {}'.format(i):agg_model
    })
    print('----------Global Model-----------')
    his = agg_model.evaluate(df_tes.to_numpy(), labels_tes)
    history.append(his)
pickle.dump(history,open('agg1','wb'))

Epoch 1/10
2/2 [==============================] - 0s 3ms/step - loss: 2.3745 - accuracy: 0.0800
Epoch 2/10
2/2 [==============================] - 0s 3ms/step - loss: 2.3204 - accuracy: 0.1000
Epoch 3/10
2/2 [==============================] - 0s 3ms/step - loss: 2.2672 - accuracy: 0.1600
Epoch 4/10
2/2 [==============================] - 0s 3ms/step - loss: 2.2191 - accuracy: 0.1800
Epoch 5/10
2/2 [==============================] - 0s 2ms/step - loss: 2.1674 - accuracy: 0.1800
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 2.1218 - accuracy: 0.2600
Epoch 7/10
2/2 [==============================] - 0s 3ms/step - loss: 2.0696 - accuracy: 0.2800
Epoch 8/10
2/2 [==============================] - 0s 3ms/step - loss: 2.0251 - accuracy: 0.2800
Epoch 9/10
2/2 [==============================] - 0s 2ms/step - loss: 1.9810 - accuracy: 0.3200
Epoch 10/10
2/2 [==============================] - 0s 3ms/step - loss: 1.9281 - accuracy: 0.3200
Epoch 1/10
2/2 [=======================

2/2 [==============================] - 0s 3ms/step - loss: 0.7830 - accuracy: 0.7400
Epoch 2/10
2/2 [==============================] - 0s 2ms/step - loss: 0.6845 - accuracy: 0.7600
Epoch 3/10
2/2 [==============================] - 0s 3ms/step - loss: 0.5790 - accuracy: 0.8200
Epoch 4/10
2/2 [==============================] - 0s 2ms/step - loss: 0.5261 - accuracy: 0.8400
Epoch 5/10
2/2 [==============================] - 0s 2ms/step - loss: 0.4982 - accuracy: 0.8400
Epoch 6/10
2/2 [==============================] - 0s 2ms/step - loss: 0.4859 - accuracy: 0.8400
Epoch 7/10
2/2 [==============================] - 0s 3ms/step - loss: 0.4675 - accuracy: 0.8400
Epoch 8/10
2/2 [==============================] - 0s 3ms/step - loss: 0.4353 - accuracy: 0.8600
Epoch 9/10
2/2 [==============================] - 0s 3ms/step - loss: 0.4030 - accuracy: 0.8800
Epoch 10/10
2/2 [==============================] - 0s 2ms/step - loss: 0.3680 - accuracy: 0.8800
Epoch 1/10
2/2 [==============================] - 

2/2 [==============================] - 0s 3ms/step - loss: 0.1337 - accuracy: 1.0000
Epoch 1/10
2/2 [==============================] - 0s 3ms/step - loss: 1.3023 - accuracy: 0.7800
Epoch 2/10
2/2 [==============================] - 0s 3ms/step - loss: 1.1809 - accuracy: 0.8000
Epoch 3/10
2/2 [==============================] - 0s 3ms/step - loss: 0.9653 - accuracy: 0.8800
Epoch 4/10
2/2 [==============================] - 0s 2ms/step - loss: 0.7880 - accuracy: 0.9000
Epoch 5/10
2/2 [==============================] - 0s 3ms/step - loss: 0.6379 - accuracy: 0.9000
Epoch 6/10
2/2 [==============================] - 0s 3ms/step - loss: 0.5000 - accuracy: 0.8800
Epoch 7/10
2/2 [==============================] - 0s 2ms/step - loss: 0.4038 - accuracy: 0.9000
Epoch 8/10
2/2 [==============================] - 0s 2ms/step - loss: 0.3166 - accuracy: 0.9200
Epoch 9/10
2/2 [==============================] - 0s 2ms/step - loss: 0.2972 - accuracy: 0.8800
Epoch 10/10
2/2 [==============================] - 

2/2 [==============================] - 0s 3ms/step - loss: 0.2123 - accuracy: 0.9400
Epoch 1/10
2/2 [==============================] - 0s 3ms/step - loss: 0.5209 - accuracy: 0.7400
Epoch 2/10
2/2 [==============================] - 0s 3ms/step - loss: 0.4190 - accuracy: 0.8400
Epoch 3/10
2/2 [==============================] - 0s 3ms/step - loss: 0.2840 - accuracy: 0.8800
Epoch 4/10
2/2 [==============================] - 0s 3ms/step - loss: 0.1972 - accuracy: 0.9400
Epoch 5/10
2/2 [==============================] - 0s 2ms/step - loss: 0.1733 - accuracy: 0.9400
Epoch 6/10
2/2 [==============================] - 0s 4ms/step - loss: 0.1642 - accuracy: 0.9200
Epoch 7/10
2/2 [==============================] - 0s 3ms/step - loss: 0.1564 - accuracy: 0.9200
Epoch 8/10
2/2 [==============================] - 0s 2ms/step - loss: 0.1514 - accuracy: 0.9400
Epoch 9/10
2/2 [==============================] - 0s 3ms/step - loss: 0.1446 - accuracy: 0.9400
Epoch 10/10
2/2 [==============================] - 

2/2 [==============================] - 0s 2ms/step - loss: 0.1048 - accuracy: 0.9600
Epoch 10/10
2/2 [==============================] - 0s 2ms/step - loss: 0.1012 - accuracy: 0.9400
Epoch 1/10
2/2 [==============================] - 0s 3ms/step - loss: 0.2358 - accuracy: 0.8800
Epoch 2/10
2/2 [==============================] - 0s 3ms/step - loss: 0.2037 - accuracy: 0.9000
Epoch 3/10
2/2 [==============================] - 0s 3ms/step - loss: 0.1514 - accuracy: 0.9400
Epoch 4/10
2/2 [==============================] - 0s 3ms/step - loss: 0.1105 - accuracy: 0.9800
Epoch 5/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0808 - accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 0s 3ms/step - loss: 0.0680 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 3ms/step - loss: 0.0621 - accuracy: 0.9800
Epoch 8/10
2/2 [==============================] - 0s 3ms/step - loss: 0.0565 - accuracy: 0.9800
Epoch 9/10
2/2 [==============================] - 

...

2/2 [==============================] - 0s 2ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 1/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0155 - accuracy: 1.0000
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0127 - accuracy: 1.0000
Epoch 3/10
2/2 [==============================] - 0s 3ms/step - loss: 0.0094 - accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0080 - accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 0s 3ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 0s 3ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 3ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 

2/2 [==============================] - 0s 3ms/step - loss: 0.0970 - accuracy: 0.9600
Epoch 8/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0693 - accuracy: 0.9600
Epoch 9/10
2/2 [==============================] - 0s 3ms/step - loss: 0.0571 - accuracy: 0.9800
Epoch 10/10
2/2 [==============================] - 0s 3ms/step - loss: 0.0514 - accuracy: 0.9800
Epoch 1/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0237 - accuracy: 0.9800
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0144 - accuracy: 1.0000
Epoch 3/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0093 - accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 0s 3ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 0s 2ms/step - loss: 8.8184e-04 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================

2/2 [==============================] - 0s 3ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 1/10
2/2 [==============================] - 0s 3ms/step - loss: 0.3103 - accuracy: 0.9400
Epoch 2/10
2/2 [==============================] - 0s 3ms/step - loss: 0.1934 - accuracy: 0.9400
Epoch 3/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0645 - accuracy: 0.9800
Epoch 4/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0192 - accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0119 - accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 

2/2 [==============================] - 0s 2ms/step - loss: 8.1918e-04 - accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 0s 2ms/step - loss: 3.0918e-04 - accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 0s 3ms/step - loss: 1.5427e-04 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 2ms/step - loss: 1.1772e-04 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 0s 2ms/step - loss: 1.1229e-04 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 2ms/step - loss: 1.1305e-04 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 0s 2ms/step - loss: 1.1691e-04 - accuracy: 1.0000
Epoch 1/10
2/2 [==============================] - 0s 3ms/step - loss: 0.2138 - accuracy: 0.9200
Epoch 2/10
2/2 [==============================] - 0s 3ms/step - loss: 0.1391 - accuracy: 0.9600
Epoch 3/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0738 - accuracy: 0.9600
Epoch 4/10
2/2 [======

2/2 [==============================] - 0s 2ms/step - loss: 0.0181 - accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0084 - accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0067 - accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 0s 3ms/step - loss: 0.0050 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0041 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 1/10
2/2 [==============================] - 0s 3ms/step - loss: 0.7848 - accuracy: 0.9600
Epoch 2/10
2/2 [==============================] - 0s 2ms/step - loss: 0.7456 - accuracy: 0.9600
Epoch 3/10
2/2 [==============================] - 

In [14]:
type(OROUND)

list